In [ ]:
import pandas as pd
# Carregando os datasets
df_2021 = pd.read_csv("./Dados Criminais 2021 - 2025/dados_2021.csv", sep=";", encoding="latin1", low_memory=False)
df_2022 = pd.read_csv("./Dados Criminais 2021 - 2025/dados_2022.csv", sep=";", encoding="latin1", low_memory=False)
df_2023 = pd.read_csv("./Dados Criminais 2021 - 2025/dados_2023.csv", sep=";", encoding="latin1", low_memory=False)
df_2024 = pd.read_csv("./Dados Criminais 2021 - 2025/dados_2024.csv", sep=";", encoding="latin1", low_memory=False)
df_2025 = pd.read_csv("./Dados Criminais 2021 - 2025/dados_2025.csv", sep=";", encoding="latin1", low_memory=False)

print("Datasets carregados:")
print("2021 ->", df_2021.shape, "linhas e colunas")
print("2022 ->", df_2022.shape, "linhas e colunas")
print("2023 ->", df_2023.shape, "linhas e colunas")
print("2024 ->", df_2024.shape, "linhas e colunas")
print("2025 ->", df_2025.shape, "linhas e colunas")

df_2022.head()


In [ ]:
# Concatenar em um único dataframe
df_crimes = pd.concat([df_2021, df_2022, df_2023, df_2024, df_2025], ignore_index=True)
print("Dataframe consolidado:", df_crimes.shape)
df_crimes.head()


In [ ]:
print("CHECKPOINT ETAPA 1 - COLETA DE DADOS")
print(f"Total de registros consolidados: {df_crimes.shape[0]:,}")
print(f"Total de colunas: {df_crimes.shape[1]}")
print("Arquivos coletados: dados_2021.csv, dados_2022.csv, dados_2023.csv, dados_2024.csv, dados_2025.csv")


In [ ]:
# Renomear colunas principais
df_crimes.rename(columns={
    "Sequência": "sequencia",
    "Data Fato": "data_fato", 
    "Hora Fato": "hora_fato",
    "Grupo Fato": "grupo_fato",
    "Tipo Enquadramento": "tipo_enquadramento",
    "Tipo Fato": "tipo_fato",
    "Municipio Fato": "municipio_fato",
    "Local Fato": "local_fato",
    "Bairro": "bairro",
    "Quantidade Vítimas": "quantidade_vitimas",
    "Idade Vítima": "idade_vitima",
    "Sexo Vítima": "sexo_vitma",
    "Cor Vítima": "cor_vitma",
}, inplace=True)
df_crimes


In [ ]:

# Análise exploratória geral
print("=== ANÁLISE EXPLORATÓRIA GERAL ===")
print(f"Total de registros: {df_crimes.shape[0]:,}")
print(f"Período dos dados: {df_crimes['data_fato'].min()} a {df_crimes['data_fato'].max()}")
print(f"Municípios únicos: {df_crimes['municipio_fato'].nunique()}")
print("\nPrincipais municípios:")
print(df_crimes['municipio_fato'].value_counts().head(10))

In [ ]:
# Remover colunas 'unnamed' geradas por excesso de separadores ou células vazias
df_crimes = df_crimes.loc[:, ~df_crimes.columns.str.lower().str.startswith('unnamed')]

In [ ]:
df_crimes.head()

In [ ]:
df_pf = df_crimes[df_crimes['municipio_fato'].str.upper() == 'PASSO FUNDO']
df_pf.head()

In [ ]:
df_pf.shape

In [ ]:
# Verificar as colunas do dataframe de Passo Fundo
print("Colunas do dataframe de Passo Fundo:")
print("=" * 40)
for i, col in enumerate(df_pf.columns, 1):
    print(f"{i:2d}. {col}")
print("=" * 40)
print(f"Total: {len(df_pf.columns)} colunas")

In [ ]:
# Remover a última coluna (vazia)
df_pf = df_pf.iloc[:, :-1]  # Remove a última coluna
print(f"Coluna vazia removida. Novo shape: {df_pf.shape}")

In [ ]:

df_pf.head()


In [ ]:
df_pf['idade_vitima'].fillna(df_pf['idade_vitima'].mean(), inplace=True)


In [ ]:

df_pf.head()


In [ ]:
import numpy as np
df_pf.replace("Sem informação", np.nan, inplace=True)
# Numéricas -> média
df_pf['idade_vitima'] = df_pf['idade_vitima'].fillna(df_pf['idade_vitima'].mean())

# Categóricas -> categoria Ignorado
df_pf['sexo_vitma'] = df_pf['sexo_vitma'].fillna("Ignorado")
df_pf['cor_vitma']  = df_pf['cor_vitma'].fillna("Ignorado")


In [ ]:

df_pf.head()


In [ ]:
# removendo duplicatas

df_pf = df_pf.drop_duplicates()
df_pf.shape


In [ ]:
# formata a data
df_pf['data_fato'] = pd.to_datetime(df_pf['data_fato'], format='%d/%m/%Y')


In [ ]:
df_pf.head()

In [ ]:
import unidecode

# Selecionar apenas colunas categóricas (object/string)
categoricas = df_pf.select_dtypes(include=['object']).columns

# Padronizar os valores
for col in categoricas:
    df_pf[col] = (
        df_pf[col]
        .astype(str)                       # garante string
        .str.strip()                       # remove espaços extras
        .str.lower()                       # tudo minúsculo
        .apply(lambda x: unidecode.unidecode(x))  # remove acentos
    )


In [ ]:
for col in categoricas:
    print(f"\nValores únicos em {col}:")
    print(df_pf[col].unique())


In [ ]:
## codigo do gpt para limpeza geral 

import unicodedata
import numpy as np

# Função para remover acentos
def remover_acentos(texto):
    if isinstance(texto, str):
        return ''.join(
            c for c in unicodedata.normalize('NFKD', texto) if not unicodedata.combining(c)
        )
    return texto

# 1. Remover duplicatas
df_pf = df_pf.drop_duplicates()

# 2. Substituir "Sem informação" por NaN em todo o DataFrame
df_pf.replace("Sem informação", np.nan, inplace=True)

# 3. Identificar colunas numéricas e categóricas
numericas = df_pf.select_dtypes(include=['int64', 'float64']).columns
categoricas = df_pf.select_dtypes(include=['object']).columns

# 4. Preencher NaN em numéricas com média
for col in numericas:
    df_pf[col] = df_pf[col].fillna(df_pf[col].mean())

# 5. Preencher NaN em categóricas com "ignorado"
for col in categoricas:
    df_pf[col] = df_pf[col].fillna("ignorado")
    # Padronizar: minúsculas, remover espaços e acentos
    df_pf[col] = df_pf[col].astype(str).str.strip().str.lower().apply(remover_acentos)

# 6. Converter colunas de data e hora para datetime
df_pf['data_fato'] = pd.to_datetime(df_pf['data_fato'], format='%d/%m/%Y', errors='coerce')
df_pf['hora_fato'] = pd.to_datetime(df_pf['hora_fato'], format='%H:%M:%S', errors='coerce').dt.time

# 7. Garantir que colunas originalmente numéricas permaneçam numéricas
for col in numericas:
    df_pf[col] = pd.to_numeric(df_pf[col], errors='coerce')

# Conferir o resultado
print("DataFrame tratado:")
print(df_pf.head())


Todas as linhas duplicadas foram removidas usando drop_duplicates().
Qualquer ocorrência de "Sem informação" foi substituída por NaN para uniformizar os valores ausentes.
Colunas numéricas (ex.: idade_vitima, quantidade_vitimas): valores ausentes foram preenchidos com a média da coluna, garantindo que não haja lacunas para cálculos estatísticos ou agregações.

Colunas categóricas (ex.: sexo_vitma, cor_vitma, tipo_fato): valores ausentes foram preenchidos com a categoria "ignorado", preservando a consistência e permitindo análises categóricas sem perder linhas.

Todas as colunas categóricas foram padronizadas:

Convertidas para minúsculas.

Espaços extras no início/fim foram removidos.

Acentos foram eliminados (Homicídio → homicidio).
Coluna data_fato convertida para datetime no formato YYYY-MM-DD.

Coluna hora_fato convertida para datetime.time, permitindo manipulação temporal correta.
Todas as colunas originalmente numéricas foram mantidas como int64 ou float64, evitando conversão involuntária para string durante a padronização.

O DataFrame agora está:

Livre de duplicatas

Sem valores ausentes ou inconsistentes

Categórico padronizado e uniforme

Datas e horas em formatos apropriados

Pronto para integração com outros datasets ou sistemas de análise

In [ ]:
import pandas as pd

caminho_meteo = r"C:\Users\Eduardo\Documents\Codes\dados-criminalidade-meteorologia\Dados meteo\passo_fundo_meteriologia.csv"

# Ler o CSV pulando as linhas de metadados
df_meteo = pd.read_csv(
    caminho_meteo, 
    sep=';', 
    decimal=',',          # para interpretar vírgula como decimal
    skiprows=9,           # pula as 9 primeiras linhas de metadados
    encoding='latin1'
)

# Conferir as primeiras linhas
print(df_meteo.head())


## as 9 primeiras linhas tem dados 
Nome: PASSO FUNDO
Codigo Estacao: A839
Latitude: -28.22666666
Longitude: -52.40361111
Altitude: 680.67
Situacao: Operante
Data Inicial: 2021-01-01
Data Final: 2025-09-05
Periodicidade da Medicao: Diaria

## irrelevantes ao meu ver

In [ ]:
print(df_meteo.columns)


In [ ]:
df_meteo.head()

In [ ]:
df_meteo.shape

In [ ]:
df_meteo = df_meteo.drop(columns=['Unnamed: 6'])


In [ ]:
df_meteo.shape

In [ ]:
# Renomear a coluna de data
df_meteo.rename(columns={'Data Medicao': 'data'}, inplace=True)

# Converter para datetime e padronizar formato YYYY-MM-DD (igual df_pf)
df_meteo['data'] = pd.to_datetime(df_meteo['data'], errors='coerce').dt.strftime('%Y-%m-%d')


In [ ]:
df_meteo.head()

In [ ]:
# 1. Garantir que a coluna de data está em datetime
df_pf['data_fato'] = pd.to_datetime(df_pf['data_fato'], errors='coerce')
df_meteo['data'] = pd.to_datetime(df_meteo['data'], errors='coerce')

# 2. Merge usando data como chave
df_merged = pd.merge(
    df_pf, 
    df_meteo, 
    left_on='data_fato', 
    right_on='data', 
    how='inner'
)

# 3. Conferir resultado
print(f"Crimes: {df_pf.shape}, Meteorologia: {df_meteo.shape}, Merge: {df_merged.shape}")
df_merged.head()


In [ ]:
# Remover a coluna 'data' duplicada (mantendo 'data_fato')
df_merged = df_merged.drop('data', axis=1)

# Verificar o resultado final
print("=== DATASET INTEGRADO ===")
print(f"Shape final: {df_merged.shape}")
print(f"Período: {df_merged['data_fato'].min()} a {df_merged['data_fato'].max()}")
print("\nPrimeiras linhas:")
df_merged.head()

In [ ]:
df_merged.isna().sum()


In [ ]:
# Verificar dados vazios no dataset integrado
print("ver dados vazios")
print("Valores nulos por coluna:")
print(df_merged.isnull().sum())

print("\nPercentual de dados vazios:")
percentual_vazios = (df_merged.isnull().sum() / len(df_merged)) * 100
print(percentual_vazios[percentual_vazios > 0])

In [ ]:
# Preencher dados meteorológicos vazios com 0
df_merged['PRECIPITACAO TOTAL, DIARIO (AUT)(mm)'].fillna(0, inplace=True)
df_merged['TEMPERATURA MAXIMA, DIARIA (AUT)(Â°C)'].fillna(20, inplace=True)  
df_merged['TEMPERATURA MINIMA, DIARIA (AUT)(Â°C)'].fillna(10, inplace=True)
df_merged['UMIDADE RELATIVA DO AR, MEDIA DIARIA (AUT)(%)'].fillna(60, inplace=True)
df_merged['VENTO, VELOCIDADE MEDIA DIARIA (AUT)(m/s)'].fillna(5, inplace=True)

print("Dados preenchidos! Verificando...")
print("Dados vazios restantes:", df_merged.isnull().sum().sum())

In [ ]:
# 2. Ver informações básicas do dataset final
print("=== INFORMAÇÕES DO DATASET FINAL ===")
print(f"Total de registros: {len(df_merged):,}")
print(f"Total de colunas: {df_merged.shape[1]}")
print(f"Período: {df_merged['data_fato'].min()} até {df_merged['data_fato'].max()}")
print("\nPrimeiras 5 linhas:")
df_merged.head()

In [ ]:
# 3. Verificar se os dados fazem sentido
print("=== VERIFICAÇÃO DE RELEVÂNCIA ===")
print("Tipos de crimes mais comuns:")
print(df_merged['tipo_fato'].value_counts().head())

print("\nEstatísticas das variáveis meteorológicas:")
colunas_meteo = ['PRECIPITACAO TOTAL, DIARIO (AUT)(mm)', 
                'TEMPERATURA MAXIMA, DIARIA (AUT)(Â°C)', 
                'TEMPERATURA MINIMA, DIARIA (AUT)(Â°C)']
print(df_merged[colunas_meteo].describe())

In [ ]:
# 4. Teste simples de correlação
print("=== TESTE DE RELEVÂNCIA ===")
print("Crimes por dia em média:", len(df_merged) / df_merged['data_fato'].nunique())
print("Dias únicos com dados:", df_merged['data_fato'].nunique())

# Ver se tem correlação básica
crimes_por_dia = df_merged.groupby('data_fato').size()
print(f"Dia com mais crimes: {crimes_por_dia.max()} crimes")
print(f"Dia com menos crimes: {crimes_por_dia.min()} crimes")

CHECKPOINT: Dataset Unificado - Criminalidade e Meteorologia
🎯 O que foi realizado:
Carregamento e Consolidação dos Dados Criminais

Carregados 5 arquivos CSV (2021-2025) com dados criminais do Rio Grande do Sul
Consolidados em um único DataFrame com renomeação padronizada das colunas
Filtrados apenas os dados de Passo Fundo
Carregamento dos Dados Meteorológicos

Carregado arquivo CSV com dados meteorológicos de Passo Fundo
Puladas as 9 primeiras linhas (metadados)
Padronização da coluna de data para formato YYYY-MM-DD
Merge dos Datasets

Integração usando a data como chave de ligação
Tipo: inner join (apenas registros com correspondência em ambos os datasets)
Resultado: Dataset unificado com informações criminais + meteorológicas
🔧 Tratamento de Colunas Vazias:

# Dados vazios identificados nas colunas meteorológicas:
# - PRECIPITACAO TOTAL, DIARIO (AUT)(mm): 2.542 valores vazios (4,3%)
# - TEMPERATURA MAXIMA, DIARIA (AUT)(°C): 1.782 valores vazios (3,0%)
# - TEMPERATURA MINIMA, DIARIA (AUT)(°C): 1.881 valores vazios (3,2%)
# - UMIDADE RELATIVA DO AR, MEDIA DIARIA (AUT)(%): 1.207 valores vazios (2,0%)
# - VENTO, VELOCIDADE MEDIA DIARIA (AUT)(m/s): 2.292 valores vazios (3,9%)

# SOLUÇÃO APLICADA - Preenchimento com valores padrão:
df_merged['PRECIPITACAO TOTAL, DIARIO (AUT)(mm)'].fillna(0, inplace=True)        # 0mm (sem chuva)
df_merged['TEMPERATURA MAXIMA, DIARIA (AUT)(Â°C)'].fillna(20, inplace=True)      # 20°C (temperatura amena)
df_merged['TEMPERATURA MINIMA, DIARIA (AUT)(Â°C)'].fillna(10, inplace=True)      # 10°C (temperatura amena)
df_merged['UMIDADE RELATIVA DO AR, MEDIA DIARIA (AUT)(%)'].fillna(60, inplace=True)  # 60% (umidade média)
df_merged['VENTO, VELOCIDADE MEDIA DIARIA (AUT)(m/s)'].fillna(5, inplace=True)       # 5 m/s (vento moderado)

In [ ]:
# Verificar resultado final
print("=== CHECKPOINT: DATASET UNIFICADO ===")
print(f"✅ Total de registros: {len(df_merged):,}")
print(f"✅ Total de colunas: {df_merged.shape[1]}")
print(f"✅ Período coberto: {df_merged['data_fato'].min()} até {df_merged['data_fato'].max()}")
print(f"✅ Dados vazios restantes: {df_merged.isnull().sum().sum()}")
print("\n🎯 O dataset agora contém:")
print("   - Informações criminais (tipo, local, vítima, etc.)")
print("   - Dados meteorológicos (temperatura, chuva, umidade, vento)")
print("   - Integração por data para análises de correlação")

Para verificar ainda e testar

# Verificar dados vazios no dataset integrado
print("ver dados vazios")
print("Valores nulos por coluna:")
print(df_merged.isnull().sum())

print("\nPercentual de dados vazios:")
percentual_vazios = (df_merged.isnull().sum() / len(df_merged)) * 100
print(percentual_vazios[percentual_vazios > 0])


# 2. Ver informações básicas do dataset final
print("=== INFORMAÇÕES DO DATASET FINAL ===")
print(f"Total de registros: {len(df_merged):,}")
print(f"Total de colunas: {df_merged.shape[1]}")
print(f"Período: {df_merged['data_fato'].min()} até {df_merged['data_fato'].max()}")
print("\nPrimeiras 5 linhas:")
df_merged.head()

# 3. Verificar se os dados fazem sentido
print("=== VERIFICAÇÃO DE RELEVÂNCIA ===")
print("Tipos de crimes mais comuns:")
print(df_merged['tipo_fato'].value_counts().head())

print("\nEstatísticas das variáveis meteorológicas:")
colunas_meteo = ['PRECIPITACAO TOTAL, DIARIO (AUT)(mm)', 
                'TEMPERATURA MAXIMA, DIARIA (AUT)(Â°C)', 
                'TEMPERATURA MINIMA, DIARIA (AUT)(Â°C)']
print(df_merged[colunas_meteo].describe())

# 4. Teste simples de correlação
print("=== TESTE DE RELEVÂNCIA ===")
print("Crimes por dia em média:", len(df_merged) / df_merged['data_fato'].nunique())
print("Dias únicos com dados:", df_merged['data_fato'].nunique())

# Ver se tem correlação básica
crimes_por_dia = df_merged.groupby('data_fato').size()
print(f"Dia com mais crimes: {crimes_por_dia.max()} crimes")
print(f"Dia com menos crimes: {crimes_por_dia.min()} crimes")